Анализ датасета

In [2]:
import pandas as pd
import sqlalchemy as db

data = pd.read_csv('D:/dataset.csv', encoding='ISO-8859-1')
data.head()

C:\Users\Honor\AppData\Local\Temp\ipykernel_6516\2397117155.py:4: DtypeWarning: Columns (4,6,31,33,61,62,63,76,79,90,92,94,96,114,115,121) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('D:/dataset.csv', encoding='ISO-8859-1')


,eventid,iyear,imonth,iday,approxdate,extended,resolution,country,country_txt,region,...,addnotes,scite1,scite2,scite3,dbsource,INT_LOG,INT_IDEO,INT_MISC,INT_ANY,related
0,197000000001,1970,7,2,NaN,0,NaN,58,Dominican Republic,2,...,NaN,NaN,NaN,NaN,PGIS,0,0,0,0,NaN
1,197000000002,1970,0,0,NaN,0,NaN,130,Mexico,1,...,NaN,NaN,NaN,NaN,PGIS,0,1,1,1,NaN
2,197001000001,1970,1,0,NaN,0,NaN,160,Philippines,5,...,NaN,NaN,NaN,NaN,PGIS,-9,-9,1,1,NaN
3,197001000002,1970,1,0,NaN,0,NaN,78,Greece,8,...,NaN,NaN,NaN,NaN,PGIS,-9,-9,1,1,NaN
4,197001000003,1970,1,0,NaN,0,NaN,101,Japan,4,...,NaN,NaN,NaN,NaN,PGIS,-9,-9,1,1,NaN


In [11]:
import plotly
import plotly.graph_objs as go
import plotly.io as pio
from plotly.subplots import make_subplots

pio.renderers.default = 'notebook' 

Количество террактов по регионам

In [15]:

# Создаем подграфики для двух круговых диаграмм
fig = make_subplots(
    rows=1, cols=2,
    specs=[[{'type': 'pie'}, {'type': 'pie'}]],
    subplot_titles=("До 2000 года", "После 2000 года")
)

# Данные до 2000 года
data_before_2000 = df[df['iyear'] < 2000]
region_counts_before_2000 = data_before_2000['region_txt'].value_counts().nlargest(5)

# Данные после 2000 года
data_after_2000 = df[df['iyear'] >= 2000]
region_counts_after_2000 = data_after_2000['region_txt'].value_counts().nlargest(5)

# Круговая диаграмма до 2000 года
fig.add_trace(
    go.Pie(
        labels=region_counts_before_2000.index,
        values=region_counts_before_2000.values,
        name="До 2000 года"
    ),
    row=1, col=1
)

# Круговая диаграмма после 2000 года
fig.add_trace(
    go.Pie(
        labels=region_counts_after_2000.index,
        values=region_counts_after_2000.values,
        name="После 2000 года"
    ),
    row=1, col=2
)

# Настраиваем заголовок и выводим графики
fig.update_layout(
    title_text="Круговые диаграммы по регионам до и после 2000 года"
)

fig.show()


Вывод: в основном большинство стран до и после 2000 года изменилось. Можно отметить сохранение и рост доли террактов в Южной Азии и Северной Африки по отношению к другим регионам.

Анализ по цели террактов (сравнение бизнес и военных целей)

In [6]:
from plotly.subplots import make_subplots

df = data[data['targtype1_txt'].isin(['Business', 'Military'])]

df1 = df[(df['iyear'] >= 1970) & (df['iyear'] < 2000)]
df2 = df[df['iyear'] >= 2000]

business_data1 = df1[df1.targtype1_txt == 'Business']
military_data1 = df1[df1.targtype1_txt == 'Military']

business_data2 = df2[df2.targtype1_txt == 'Business']
military_data2 = df2[df2.targtype1_txt == 'Military']

years1 = df1['iyear'].unique()
years2 = df2['iyear'].unique()

business_count1 = business_data1.groupby('iyear').size().reindex(years1, fill_value=0)
military_count1 = military_data1.groupby('iyear').size().reindex(years1, fill_value=0)

business_count2 = business_data2.groupby('iyear').size().reindex(years2, fill_value=0)
military_count2 = military_data2.groupby('iyear').size().reindex(years2, fill_value=0)

fig = make_subplots(rows=2, cols=1, subplot_titles=("1970-2000", "2000-2017"))

trace1 = go.Bar(
    x=years1,
    y=business_count1,
    name="Business",
    marker=dict(color='rgba(171, 50, 96, 0.6)')
)
trace2 = go.Bar(
    x=years1,
    y=military_count1,
    name="Military",
    marker=dict(color='rgba(12, 50, 196, 0.6)')
)
fig.add_trace(trace1, row=1, col=1)
fig.add_trace(trace2, row=1, col=1)

trace3 = go.Bar(
    x=years2,
    y=business_count2,
    marker=dict(color='rgba(171, 50, 96, 0.6)')
)
trace4 = go.Bar(
    x=years2,
    y=military_count2,
    marker=dict(color='rgba(12, 50, 196, 0.6)')
)
fig.add_trace(trace3, row=2, col=1)
fig.add_trace(trace4, row=2, col=1)


fig.update_layout(
    barmode='overlay',
    title_text="Number of incidents")

Вывод: До 80 года можно наблюдать преобладание бизнес целей над военными и в целом до 2000 год чаще бизнес цели преобладали над военными (почти 2 к 1), а после 2000 наоборот чаще военные являются преобладающими. Последние шесть лет еще и сбольшим разрывом.

Статистика по видам использованного оружия по годам 

In [25]:
df = data[data['weaptype1_txt'].isin(['Explosives', 'Firearms',
                                     'Unknown', 'Incendiary'])]

df_exp = df[df.weaptype1_txt == 'Explosives']
df_fir = df[df.weaptype1_txt == 'Firearms']
df_unk = df[df.weaptype1_txt == 'Unknown']
df_inc = df[df.weaptype1_txt == 'Incendiary']

years = df['iyear'].unique()

exp_count = df_exp.groupby('iyear').size().reindex(years, fill_value=0)
fir_count = df_fir.groupby('iyear').size().reindex(years, fill_value=0)
unk_count = df_unk.groupby('iyear').size().reindex(years, fill_value=0)
inc_count = df_inc.groupby('iyear').size().reindex(years, fill_value=0)

# creating trace1
trace1 =go.Scatter(
                    x = years,
                    y = exp_count,
                    mode = "markers",
                    name = "Explosives",
                    marker = dict(color = 'rgba(255, 128, 255, 0.8)'),
                    text= exp_count)
# creating trace2
trace2 =go.Scatter(
                    x = years,
                    y = fir_count,
                    mode = "markers",
                    name = "Firearms",
                    marker = dict(color = 'rgba(255, 128, 2, 0.8)'),
                    text= fir_count)
# creating trace3
trace3 =go.Scatter(
                    x = years,
                    y = unk_count,
                    mode = "markers",
                    name = "Unknown",
                    marker = dict(color = 'rgba(0, 255, 200, 0.8)'),
                    text= unk_count)

trace4 =go.Scatter(
                    x = years,
                    y = inc_count,
                    mode = "markers",
                    name = "Incendiary",
                    marker = dict(color = 'rgba(0, 0, 139, 0.8)'),
                    text= inc_count)
data_ = [trace1, trace2, trace3, trace4]
layout = dict(title = 'Number of types of weapons used',
              xaxis= dict(title= 'Year',ticklen= 5,zeroline= False),
              yaxis= dict(title= 'Quantity',ticklen= 5,zeroline= False)
             )
fig = dict(data = data_, layout = layout)
fig = go.Figure(fig)
fig.show()

Вывод: начиная с 1980 года наибольшей "популярностью" пользовалось взрывчатое и огнестрельное оружие, рост разрыва начал появлятся начиная с 2005 года и достиг пика 2014.

In [17]:
data['weaptype1_txt'].value_counts().nlargest(5)


weaptype1_txt
Explosives    92426
Firearms      58524
Unknown       15157
Incendiary    11135
Melee          3655
Name: count, dtype: int64

Анализ убытков от террактов

In [36]:
df = data[data['propextent_txt'].isin(['Minor (likely < $1 million)',
       'Major (likely >= $1 million but < $1 billion)', 'Unknown',
       'Catastrophic (likely >= $1 billion)'])]

categ = data['propextent_txt'].unique()
prop_count = df.groupby('propextent_txt').size().reindex(categ, fill_value=0)
          
trace1 = {
  'x': categ,
  'y': prop_count,
  'name': 'citation',
  'type': 'bar'
};

data_ = [trace1];
layout = {
  'barmode': 'relative',
  'title': 'Losses from terrorist attacks'
};
fig = go.Figure(data = data_, layout = layout)
fig.show()

Вывод: чаще всего отмечаются незначительные убытки

In [34]:
prop_count

propextent_txt
NaN                                                  0
Minor (likely < $1 million)                      43304
Major (likely >= $1 million but < $1 billion)      909
Unknown                                          19846
Catastrophic (likely >= $1 billion)                  6
dtype: int64